In [1]:
import cv2
import numpy as np

# Load the image in grayscale
image = cv2.imread('city_view.png', cv2.IMREAD_GRAYSCALE)

threshold = 240  # Adjust this value as needed

# Create a 2D matrix of the same size as the image
height, width = image.shape
result_matrix = np.zeros((height, width), dtype=np.int32)

# Fill the cells based on the threshold
for i in range(height):
    for j in range(width):
        if image[i, j] < threshold:
            result_matrix[i, j] = 0
        else:
            result_matrix[i, j] = -1

#draw_matrix(result_matrix)
print("done loading image")

done loading image


In [2]:
import queue

def mark_pixel(x, y, value, pixel_queue):
    if x < 0 or y < 0 or x >= height or y >= width:
        return
    
    if result_matrix[x, y] != -1:
        return
    
    result_matrix[x, y] = value
    pixel_queue.put([x,y])

# Create a FIFO queue
pixel_queue = queue.Queue()

# enqueue all walls (v==0)

for i in range(height):
    for j in range(width):
        if result_matrix[i, j] == 0:
            pixel_queue.put([i, j])
            

# process each pixel
while not pixel_queue.empty():
    # print(pixel_queue.qsize(), " || ", i, "-", j)
    [i, j] = pixel_queue.get()
    current_value = result_matrix[i, j] + 1
    mark_pixel(i-1, j, current_value, pixel_queue)
    mark_pixel(i+1, j, current_value, pixel_queue)
    mark_pixel(i, j-1, current_value, pixel_queue)
    mark_pixel(i, j+1, current_value, pixel_queue)
    

print("done lableing distance to wall")

done lableing distance to wall


In [3]:
def create_2_matrix(result_matrix):
    new_matrix = np.empty_like(result_matrix, dtype=np.str_)
    door_matrix = result_matrix.copy()
    height, width = image.shape
    for i in range(height):
        for j in range(width):
            current_value = result_matrix[i, j]

            # Get the values of adjacent cells
            up_value = result_matrix[i - 1, j] if i > 0 else None
            down_value = result_matrix[i + 1, j] if i < height - 1 else None
            left_value = result_matrix[i, j - 1] if j > 0 else None
            right_value = result_matrix[i, j + 1] if j < width - 1 else None

            if current_value == 0:
                new_matrix[i, j] = '0'
            elif (up_value is None or current_value >= up_value) and \
                 (down_value is None or current_value >= down_value) and \
                 (left_value is None or current_value >= left_value) and \
                 (right_value is None or current_value >= right_value):
                new_matrix[i, j] = '*'
            else:
                new_matrix[i, j] = '.'
    get_map_doors(height, width, new_matrix, door_matrix)
    return new_matrix, door_matrix
    
def change_door_x(path_clear, path, new_matrix, door_matrix):
    if path_clear:
        for x, y in path:
            new_matrix[x, y] = 'X'
            door_matrix[x, y] = 0
            

def get_map_doors(height, width, new_matrix, door_matrix):
    threshold = min(height, width)/10
    for i in range(height):
        for j in range(width):
            if new_matrix[i, j] == '0':
                path = []
                path_clear = True
                count = 0
                
                # Check up
                x = i - 1
                while x >= 0 and new_matrix[x, j] != '0':
                    if new_matrix[x, j] == '.':
                        path.append((x, j))
                    else:
                        path = []
                        path_clear = False
                        break
                    x -= 1
                    if x < 0 or count == threshold:
                        path = []
                        path_clear = False
                        break
                    if len(path) > 0:
                        path_clear = True
                    count += 1
                
                change_door_x(path_clear, path, new_matrix, door_matrix)
                count = 0

                # Check down
                x = i + 1
                while x < height and new_matrix[x, j] != '0':
                    if new_matrix[x, j] == '.':
                        path.append((x, j))
                    else:
                        path = []
                        path_clear = False
                        break
                    x += 1
                    if x >= height  or count == threshold:
                        path = []
                        path_clear = False
                        break
                    if len(path) > 0:
                        path_clear = True
                    count += 1

                change_door_x(path_clear, path, new_matrix, door_matrix)
                count = 0

                # Check left
                y = j - 1
                while y >= 0 and new_matrix[i, y] != '0':
                    if new_matrix[i, y] == '.':
                        path.append((i, y))
                    else:
                        path = []
                        path_clear = False
                        break
                    y -= 1
                    if y < 0  or count == threshold:
                        path = []
                        path_clear = False
                        break
                    if len(path) > 0:
                        path_clear = True
                    count += 1

                change_door_x(path_clear, path, new_matrix, door_matrix)
                count = 0

                # Check right
                y = j + 1
                while y < width and new_matrix[i, y] != '0':
                    if new_matrix[i, y] == '.':
                        path.append((i, y))
                    else:
                        path = []
                        path_clear = False
                        break
                    y += 1
                    if y >= width  or count == threshold:
                        path = []
                        path_clear = False
                        break
                    if len(path) > 0:
                        path_clear = True
                        count += 1

                change_door_x(path_clear, path, new_matrix, door_matrix)
                
door_matrix, new_matrix = create_2_matrix(result_matrix)
print("done door identification and room seperation")

done door identification and room seperation


In [4]:
#print_matrix(result_matrix)
#print("================")
#print_matrix(new_matrix)

In [5]:
def draw_matrix(image):
    new_img = np.array(image.copy())
    # Convert -1 values to 255 (white) for visualization
    new_img[new_img == -1] = 127
    
    new_img = new_img.astype(np.uint8)

    print(new_img.shape)
    # Combine the mask with the original image to draw the cells
    cv2.imshow("Grayscale Image", new_img)
    cv2.waitKey(0)  # This line allows you to close the image window by pressing any key
    cv2.destroyAllWindows()
    
def print_matrix(matrix):
    height, width = image.shape
    for i in range(height):
        for j in range(width):
            print(matrix[i, j], end=' ')
        print()  # Move to the next row

In [6]:
import random
import time
import copy


pivot_count = 0
def random_color():
    red = np.int32(random.randint(150, 255))
    green = np.int32(random.randint(150, 255))
    blue = np.int32(random.randint(150, 255))
    return (red, green, blue)

def is_vaild_index(index, verify_painted_matrix):
    [x, y] = index
    num_rows = len(verify_painted_matrix)
    num_cols = len(verify_painted_matrix[0])
    
    if not (0 <= x < num_rows and 0 <= y < num_cols):
        return False
    
    if verify_painted_matrix[x][y] == True:
        return False
    
    # in range, not painted
    return True
    
def is_not_all_painted(verify_painted_matrix):
    for row in verify_painted_matrix:
        for value in row:
            if value == False:
                return True
            
    return False

def find_max(distance_matrix, verify_painted_matrix):
    max_val = 0
    cor = [-1, -1]
    
    for i in range(len(distance_matrix)):
        for j in range(len(distance_matrix[0])):
            if verify_painted_matrix[i][j] == False:
                # not paint
                if distance_matrix[i][j] > max_val:
                    max_val = distance_matrix[i][j]
                    cor = [i, j]
    
    return cor

def updateMinMax(x, y, min_x, max_x, min_y, max_y):
    if x < min_x:
        min_x = x
    elif x > max_x:
        max_x = x
    if y < min_y:
        min_y = y
    elif y > max_y:
        max_y = y
        
    return min_x, max_x, min_y, max_y

def paint_room(cordinate, distance_matrix, verify_painted_matrix, colored_image, count):
    # print("paint room of: ",cordinate)
    [x, y] = cordinate
    
    color = random_color()
    
    room_pixel_queue = queue.Queue()
    room_pixel_queue.put(cordinate)
    
    r_val = 50 + count * 20
    if r_val > 255:
        r_val = 255
    colored_image[x][y] = (np.int32(0), np.int32(0), np.int32(r_val))
    verify_painted_matrix[x][y] = True
    count = 0
    
    min_x, max_x, min_y, max_y = 0, 0, 0, 0
    while not room_pixel_queue.empty():
        cor = room_pixel_queue.get()
        # print(cor)
        x = cor[0]
        y = cor[1]
        
        min_x, max_x, min_y, max_y = updateMinMax(x, y, min_x, max_x, min_y, max_y)
        
        curent_value = distance_matrix[x][y]
        count = count + 1
        
        x_n = x -1
        y_n = y
        if is_vaild_index([x_n, y_n], verify_painted_matrix) and distance_matrix[x_n][y_n] != 0:
            # able to paint
            colored_image[x_n][y_n] = color
            verify_painted_matrix[x_n][y_n] = True
            room_pixel_queue.put([x_n, y_n])
                
        x_n = x + 1
        y_n = y
        if is_vaild_index([x_n, y_n], verify_painted_matrix) and distance_matrix[x_n][y_n] != 0:
            # able to paint
            colored_image[x_n][y_n] = color
            verify_painted_matrix[x_n][y_n] = True
            room_pixel_queue.put([x_n, y_n])
                
        x_n = x
        y_n = y - 1
        if is_vaild_index([x_n, y_n], verify_painted_matrix) and distance_matrix[x_n][y_n] != 0:
            # able to paint
            colored_image[x_n][y_n] = color
            verify_painted_matrix[x_n][y_n] = True
            room_pixel_queue.put([x_n, y_n])
                
        x_n = x
        y_n = y + 1
        if is_vaild_index([x_n, y_n], verify_painted_matrix) and distance_matrix[x_n][y_n] != 0:
            # able to paint
            colored_image[x_n][y_n] = color
            verify_painted_matrix[x_n][y_n] = True
            room_pixel_queue.put([x_n, y_n])
    
    # print(count)
        

def create_colored_room(distance_matrix, colored_image):
    height = len(distance_matrix)
    width = len(distance_matrix[0])
    
    verify_painted_matrix = [[False for _ in range(width)] for _ in range(height)]
    
    paint_room([1,1], distance_matrix, verify_painted_matrix, colored_image, 0)
    
    outside_building = copy.deepcopy(verify_painted_matrix)
                
    count = 0
    while is_not_all_painted(verify_painted_matrix):
        # find max
        cordinate = find_max(distance_matrix, verify_painted_matrix)
        if cordinate == [-1, -1]:
            break
            
        paint_room(cordinate, distance_matrix, verify_painted_matrix, colored_image, count)
        count = count + 1
        
    
    # print("all room done")
    return outside_building
    
    # locate the higest value

In [7]:
pivot_count = 0
colored_image = np.zeros((height, width, 3), dtype=np.uint8)
ouside_building_mask = create_colored_room(new_matrix, colored_image)

# print("das")
print(colored_image.shape)
print(colored_image[0][0])

(521, 1651, 3)
[240 204 175]


In [8]:
def mark_path(result_matrix, colored_image):
    height = len(result_matrix)
    width = len(result_matrix[0])
    
    for i in range(height):
        for j in range(width):
            current_value = result_matrix[i, j]

            # Get the values of adjacent cells
            up_value = result_matrix[i - 1, j] if i > 0 else None
            down_value = result_matrix[i + 1, j] if i < height - 1 else None
            left_value = result_matrix[i, j - 1] if j > 0 else None
            right_value = result_matrix[i, j + 1] if j < width - 1 else None

            if (up_value is None or current_value >= up_value) and \
               (down_value is None or current_value >= down_value) and \
               (left_value is None or current_value >= left_value) and \
               (right_value is None or current_value >= right_value):
                colored_image[i][j] = (np.int32(0), np.int32(0), np.int32(0))

In [9]:

#ori_Img = image
#mark_path(result_matrix, ori_Img)
#cv2.imwrite("floor_path.png", ori_Img)

In [10]:
#print_matrix(new_matrix)

In [11]:
# get path way
def local_highest(door_matrix, new_matrix):
    entry_matrix = new_matrix.copy()
    height, width = door_matrix.shape
    for i in range(height):
        for j in range(width):
            if door_matrix[i, j] == '*':
                entry_matrix[i, j] = 1
            else:
                entry_matrix[i, j] = 0
    
    return entry_matrix

In [12]:
path_matrix = local_highest(door_matrix, new_matrix)

def mark_path(path_matrix, colored_image):
    path_image = colored_image.copy()
    height, width = path_matrix.shape
    for i in range(height):
        for j in range(width):
            if path_matrix[i][j] == 1:
                path_image[i][j] = (np.int32(240), np.int32(0), np.int32(0))
            
    return path_image


path_image = mark_path(path_matrix, colored_image)

cv2.imwrite("floorPlan_0_1.png", path_image)

True

In [13]:
import queue

def mark_pixel(x, y, value, pixel_queue):
    if x < 0 or y < 0 or x >= height or y >= width:
        return
    
    if new_matrix[x, y] != -1:
        return
    
    new_matrix[x, y] = value
    pixel_queue.put([x,y])

# Create a FIFO queue
pixel_queue = queue.Queue()

# enqueue all walls (v==0)

for i in range(height):
    for j in range(width):
        if new_matrix[i, j] == 0:
            pixel_queue.put([i, j])
        else:
            new_matrix[i, j] = -1
            

# process each pixel
while not pixel_queue.empty():
    # print(pixel_queue.qsize(), " || ", i, "-", j)
    [i, j] = pixel_queue.get()
    current_value = new_matrix[i, j] + 1
    mark_pixel(i-1, j, current_value, pixel_queue)
    mark_pixel(i+1, j, current_value, pixel_queue)
    mark_pixel(i, j-1, current_value, pixel_queue)
    mark_pixel(i, j+1, current_value, pixel_queue)
    

print("done lableing distance to wall")

done lableing distance to wall


In [14]:
test_door, test_new_matrix = create_2_matrix(new_matrix)
test_path_matrix = local_highest(test_door, test_new_matrix)

def mark_path(path_matrix, colored_image):
    path_image = colored_image.copy()
    height, width = path_matrix.shape
    for i in range(height):
        for j in range(width):
            if path_matrix[i][j] == 1:
                path_image[i][j] = (np.int32(240), np.int32(0), np.int32(0))
            
    return path_image


path_image = mark_path(test_path_matrix, colored_image)

cv2.imwrite("city_0_test2.png", path_image)

True

In [15]:
# remove outside building path
def apply_mask(mask, matrix, value):
    result_m = matrix.copy()
    height, width = len(mask), len(mask[0])
    for i in range(height):
        for j in range(width):
            if mask[i][j] == 1 or mask[i][j]:
                result_m[i][j] = value
            
    return result_m

test_path_matrix = apply_mask(ouside_building_mask, test_path_matrix, 0)
out_image = mark_path(test_path_matrix, colored_image)

cv2.imwrite("city_0_notOut.png", out_image)

True

In [16]:
# remove doors's blocking
def create_door_mask(input_matrix):
    result_m = [[False for _ in row] for row in input_matrix]

    for i in range(len(input_matrix)):
        for j in range(len(input_matrix[i])):
            if input_matrix[i][j] == 'X':
                result_m[i][j] = True

    return result_m

door_mask = create_door_mask(door_matrix)
test_new_matrix = apply_mask(door_mask, test_new_matrix, 1)
open_door_img = apply_mask(door_mask, out_image, (np.int32(255), np.int32(255), np.int32(255)))
cv2.imwrite("city_0_openDoor.png", open_door_img)

True

In [17]:
# locate isoloated space and remove path in there
# create mask for accessable area
def checkDoor(cor, door_mask):
    [x, y] = cor
    return door_mask[x][y]

def mark_area(cordinate, distance_matrix, visited, door_mask):
    height = len(distance_matrix)
    width = len(distance_matrix[0])
    current_area = [[False for _ in range(width)] for _ in range(height)]
    
    # print("paint room of: ",cordinate)
    contain_door = False
    [x, y] = cordinate
    
    visited[x][y] = True
    current_area[x][y] = True
    room_pixel_queue = queue.Queue()
    room_pixel_queue.put(cordinate)
    
    while not room_pixel_queue.empty():
        cor = room_pixel_queue.get()
        # print(cor)
        x = cor[0]
        y = cor[1]
        
        x_n = x -1
        y_n = y
        if is_vaild_index([x_n, y_n], visited) and distance_matrix[x_n][y_n] != 0:
            contain_door = contain_door or checkDoor([x_n, y_n], door_mask)
            visited[x_n][y_n] = True
            current_area[x_n][y_n] = True
            room_pixel_queue.put([x_n, y_n])
                
        x_n = x + 1
        y_n = y
        if is_vaild_index([x_n, y_n], visited) and distance_matrix[x_n][y_n] != 0:
            contain_door = contain_door or checkDoor([x_n, y_n], door_mask)
            visited[x_n][y_n] = True
            current_area[x_n][y_n] = True
            room_pixel_queue.put([x_n, y_n])
                
        x_n = x
        y_n = y - 1
        if is_vaild_index([x_n, y_n], visited) and distance_matrix[x_n][y_n] != 0:
            contain_door = contain_door or checkDoor([x_n, y_n], door_mask)
            visited[x_n][y_n] = True
            current_area[x_n][y_n] = True
            room_pixel_queue.put([x_n, y_n])
                
        x_n = x
        y_n = y + 1
        if is_vaild_index([x_n, y_n], visited) and distance_matrix[x_n][y_n] != 0:
            contain_door = contain_door or checkDoor([x_n, y_n], door_mask)
            visited[x_n][y_n] = True
            current_area[x_n][y_n] = True
            room_pixel_queue.put([x_n, y_n])
            
        return contain_door, current_area
            
def creat_accessable_mask(test_new_matrix, door_mask):
    is_accessable = False
    
    height = len(door_mask)
    width = len(door_mask[0])
    visited = [[False for _ in range(width)] for _ in range(height)]
    
    while not is_accessable:
        accessable_mask = [[False for _ in range(width)] for _ in range(height)]
        cordinate = find_max(new_matrix, visited)
        print(cordinate)
        
        is_accessable, current_mask = mark_area(cordinate, new_matrix, visited, door_mask)
        if is_accessable:
            return current_mask    

In [ ]:
accessable_mask = creat_accessable_mask(test_new_matrix, door_mask)

[0, 0]
[0, 2]
[1, 1]
[2, 0]
[0, 4]
[1, 3]
[2, 2]
[3, 1]
[4, 0]
[0, 6]
[1, 5]
[2, 4]
[3, 3]
[4, 2]
[5, 1]
[6, 0]
[0, 8]
[1, 7]
[2, 6]
[3, 5]
[4, 4]
[5, 3]
[6, 2]
[7, 1]
[8, 0]
[0, 10]
[1, 9]
[2, 8]
[3, 7]
[4, 6]
[5, 5]
[6, 4]
[7, 3]
[8, 2]
[9, 1]
[10, 0]
[0, 12]
[1, 11]
[2, 10]
[3, 9]
[4, 8]
[5, 7]
[6, 6]
[7, 5]
[8, 4]
[9, 3]
[10, 2]
[11, 1]
[12, 0]
[0, 14]
[1, 13]
[2, 12]
[3, 11]
[4, 10]
[5, 9]
[6, 8]
[7, 7]
[8, 6]
[9, 5]
[10, 4]
[11, 3]
[12, 2]
[13, 1]
[14, 0]
[0, 16]
[1, 15]
[2, 14]
[3, 13]
[4, 12]
[5, 11]
[6, 10]
[7, 9]
[8, 8]
[9, 7]
[10, 6]
[11, 5]
[12, 4]
[13, 3]
[14, 2]
[15, 1]
[16, 0]
[0, 18]
[1, 17]
[2, 16]
[3, 15]
[4, 14]
[5, 13]
[6, 12]
[7, 11]
[8, 10]
[9, 9]
[10, 8]
[11, 7]
[12, 6]
[13, 5]
[14, 4]
[15, 3]
[16, 2]
[17, 1]
[18, 0]
[0, 20]
[1, 19]
[2, 18]
[3, 17]
[4, 16]
[5, 15]
[6, 14]
[7, 13]
[8, 12]
[9, 11]
[10, 10]
[11, 9]
[12, 8]
[13, 7]
[14, 6]
[15, 5]
[16, 4]
[17, 3]
[18, 2]
[19, 1]
[20, 0]
[0, 22]
[1, 21]
[2, 20]
[3, 19]
[4, 18]
[5, 17]
[6, 16]
[7, 15]
[8, 14]
[9, 13]
[